#  Single Neuron and Activation Functions

This example shows how to calculate the the output of a single neuron using different activation functions.

Farhad kamangar 2026 (with the help from chatgpt V 5.2)

In [1]:
# installation and import code cell
!pip -q install -U plotly
!pip -q install "ipywidgets>=7.7,<9"
from google.colab import output
output.enable_custom_widget_manager()

import ipywidgets as widgets
from IPython.display import display, Math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from ipywidgets import (
    FloatSlider, Dropdown, Checkbox, Button,
    HBox, VBox, Layout, interactive_output, Label
)
from IPython.display import display

# --- Colab widget support (required for ipywidgets to render & update in Colab) ---
try:
    from google.colab import output as colab_output
    colab_output.enable_custom_widget_manager()
except Exception:
    pass
try:
    import google.colab  # noqa: F401
    pio.renderers.default = "colab"
except Exception:
    # notebook_connected works well in many Jupyter environments
    pio.renderers.default = "notebook_connected"

In [2]:
# import numpy as np
# from IPython.display import Math, display

def display_named_matrix(items, *, default_sep=r"\qquad"):
    # """
    # items: dict mapping name_latex -> entry_spec

    # entry_spec must be a dict with at least:
    #   {"value": <scalar|array|torch.Tensor|sympy expr|sympy Matrix>}

    # Optional per-entry keys:
    #   - precision: int (default 3)
    #   - transpose: bool (default False)
    #   - one_d_as: "row"|"column" (default "column")  # only for 1-D vectors
    #   - transpose_symbol: LaTeX (default r"^{\mathsf{T}}")
    #   - sep_after: LaTeX separator after this item (default_sep if not last)
    # """

    # SymPy optional
    # usage example
    # A = np.array([[0.12, -0.30],
    #           [0.80,  0.20]])
    # B = np.array([0.12, -0.30])
    # C = np.array([[0.12, -0.30],
    #               [0.80,  0.20]])
    # display_named_matrix({
    #     r"\mathbf{A}": {"value": A, "precision": 2, "transpose": True},
    #     r"\mathbf{B}": {"value": B, "precision": 4, "one_d_as": "column"},
    #     r"\alpha":     {"value": 3.14159, "precision": 5},
    #     r"\mathbf{C}": {"value": C, "transpose": True, "one_d_as": "row", "transpose_symbol": r"^{\top}"},
    # })

    try:
        import sympy as sp
        _HAS_SYMPY = True
    except ImportError:
        sp = None
        _HAS_SYMPY = False

    def to_numpy(x):
        try:
            import torch
            if isinstance(x, torch.Tensor):
                return x.detach().cpu().numpy()
        except ImportError:
            pass
        return x

    def is_sympy_obj(x):
        return _HAS_SYMPY and isinstance(x, sp.Basic)

    def is_sympy_matrix(x):
        return _HAS_SYMPY and isinstance(x, sp.MatrixBase)

    def format_entry(v, precision):
        if is_sympy_obj(v):
            return sp.latex(v)
        try:
            return f"{float(v):.{precision}f}"
        except Exception:
            return str(v)

    def render_one(name_latex, spec):
        if not isinstance(spec, dict) or "value" not in spec:
            raise ValueError(f"Entry for '{name_latex}' must be a dict with a 'value' key.")

        x = to_numpy(spec["value"])
        precision = int(spec.get("precision", 3))
        tr = bool(spec.get("transpose", False))
        one_d_as = str(spec.get("one_d_as", "column")).lower()
        transpose_symbol = spec.get("transpose_symbol", r"^{\mathsf{T}}")

        nm = rf"{name_latex}{transpose_symbol}" if tr else name_latex

        # ---- SymPy scalar ----
        if is_sympy_obj(x) and not hasattr(x, "shape"):
            return rf"\begin{{array}}{{c c c}} {nm} & = & {sp.latex(x)} \end{{array}}"

        # ---- Numeric scalar / 0-D ----
        if np.isscalar(x) or (isinstance(x, np.ndarray) and x.shape == ()):
            return rf"\begin{{array}}{{c c c}} {nm} & = & {format_entry(x, precision)} \end{{array}}"

        # ---- SymPy matrix -> object ndarray ----
        if is_sympy_matrix(x):
            x = np.array(x.tolist(), dtype=object)

        x = np.asarray(x, dtype=object)

        # 1×1 -> scalar
        if x.ndim == 0 or (x.ndim == 2 and x.shape == (1, 1)):
            return rf"\begin{{array}}{{c c c}} {nm} & = & {format_entry(np.squeeze(x), precision)} \end{{array}}"

        # ---- Orientation & transpose ----
        if x.ndim == 1:
            # 1-D transpose is a no-op; implement semantic transpose by flipping display orientation
            orient = one_d_as
            if tr:
                orient = "row" if orient == "column" else "column"
            x2 = x.reshape(1, -1) if orient == "row" else x.reshape(-1, 1)
        else:
            x2 = x if x.ndim == 2 else x.reshape(x.shape[0], -1)
            if tr:
                x2 = x2.T

        rows = [" & ".join(format_entry(v, precision) for v in row) for row in x2]
        mat = r"\begin{bmatrix}" + r" \\ ".join(rows) + r"\end{bmatrix}"

        return rf"\begin{{array}}{{c c c}} {nm} & = & \vcenter{{{mat}}} \end{{array}}"

    # ---- Build full line with per-entry separators ----
    parts = []
    keys = list(items.keys())
    for i, k in enumerate(keys):
        frag = render_one(k, items[k])
        parts.append(frag)

        if i < len(keys) - 1:
            sep_after = items[k].get("sep_after", default_sep)
            parts.append(sep_after)

    display(Math("".join(parts)))


In [22]:
INTERACTIVE_3D = True  # Plotly is interactive in Colab/Jupyter by default
def identity(net): return net
def d_identity(net): return np.ones_like(net)

def sigmoid(net):
    return 1 / (1 + np.exp(-net))
def d_sigmoid(net):
    s = sigmoid(net)
    return s * (1 - s)

def tanh(net): return np.tanh(net)
def d_tanh(net):
    y = np.tanh(net)
    return 1 - y**2

def relu(net): return np.maximum(0, net)
def d_relu(net): return (net > 0).astype(float)

def leaky_relu(net, leaky_alpha=0.1):
    return np.where(net >= 0, net, leaky_alpha * net)
def d_leaky_relu(net, leaky_alpha=0.1):
    return np.where(net >= 0, 1.0, leaky_alpha)

def hard_limit(net):
    return np.where(net >= 0, 1.0, 0.0)
def d_hard_limit(net):
    # Not differentiable; use 0 almost everywhere (subgradient convention)
    return np.zeros_like(net, dtype=float)
def symetrical_hard_limit(net):
    return np.where(net >= 0, 1.0, -1)
def d_symetrical_hard_limit(net):
    # Not differentiable; use 0 almost everywhere (subgradient convention)
    return np.zeros_like(net, dtype=float)
def softplus(net):
    return np.log1p(np.exp(-np.abs(net))) + np.maximum(net, 0)
def d_softplus(net):
    return sigmoid(net)

ACTS = {
    "identity": (identity, d_identity),
    "sigmoid": (sigmoid, d_sigmoid),
    "tanh": (tanh, d_tanh),
    "ReLU": (relu, d_relu),
    "leaky ReLU": (leaky_relu, d_leaky_relu),
    "hard_limit": (hard_limit, d_hard_limit),
    "symmetrical_hard_limit": (symetrical_hard_limit, d_symetrical_hard_limit),
    "softplus": (softplus, d_softplus),
}

def apply_act(net, activation, leaky_alpha=0.1):
    f, _ = ACTS[activation]
    if activation == "leaky ReLU":
        return f(net, leaky_alpha=leaky_alpha)
    return f(net)

def apply_dact(net, activation, leaky_alpha=0.1):
    _, df = ACTS[activation]
    if activation == "leaky ReLU":
        return df(net, leaky_alpha=leaky_alpha)
    return df(net)

# =========================
# Decision boundary points for net=0: w1*x1 + w2*x2 + b = 0
# Returns (xs, ys) or None
# =========================
def decision_boundary_line(w1, w2, b, xlim=(-2, 2), n=400):
    eps = 1e-9
    xmin, xmax = xlim

    if abs(w2) > eps:
        xs = np.linspace(xmin, xmax, n)
        ys = (-b - w1 * xs) / w2
        return xs, ys
    else:
        if abs(w1) > eps:
            x_vert = (-b) / w1
            ys = np.linspace(xmin, xmax, n)
            xs = np.full_like(ys, x_vert)
            return xs, ys
        else:
            return None

def plot_output_surface_plotly(
    w1, w2, b, activation, leaky_alpha,
    x1_current, x2_current, a_current,
    show_boundary=True,
    grid_n=70,
    xlim=(-2, 2), ylim=(-2, 2)
):
    # Grid
    x1g = np.linspace(xlim[0], xlim[1], int(grid_n))
    x2g = np.linspace(ylim[0], ylim[1], int(grid_n))
    X1, X2 = np.meshgrid(x1g, x2g)
    NET = w1 * X1 + w2 * X2 + b
    A = apply_act(NET, activation, leaky_alpha=leaky_alpha)

    # Boundary points (net=0)
    bx = by = bz = None
    if show_boundary:
        line = decision_boundary_line(w1, w2, b, xlim=xlim, n=500)
        if line is not None:
            bx, by = line
            bnet = w1 * bx + w2 * by + b
            bz = apply_act(bnet, activation, leaky_alpha=leaky_alpha)

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "surface"}, {"type": "contour"}]],
        column_widths=[0.58, 0.42],
        horizontal_spacing=0.06,
        subplot_titles=(
            "3D output surface a(x1,x2)",
            "2D contour of a(x1,x2)"
        )
    )

    # Surface
    fig.add_trace(
        go.Surface(x=X1, y=X2, z=A, showscale=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Surface(x=X1, y=X2, z=np.zeros_like(A),opacity=0.5, showscale=False),
        row=1, col=1
    )
    # Current point
    fig.add_trace(
        go.Scatter3d(
            x=[x1_current], y=[x2_current], z=[a_current],
            mode="markers",
            marker=dict(size=6),
            name="current (x1,x2)"
        ),
        row=1, col=1
    )

    # Boundary curve lifted onto surface
    if bx is not None:
        fig.add_trace(
            go.Scatter3d(
                x=bx, y=by, z=np.zeros_like(bz),
                mode="lines",
                line=dict(width=6),
                name="boundary net=0"
            ),
            row=1, col=1
        )

    # Contour plot
    fig.add_trace(
        go.Contour(
            x=x1g, y=x2g, z=A,
            contours=dict(showlabels=False),
            colorbar=dict(title="a", len=0.75),
            name="a(x1,x2)"
        ),
        row=1, col=2
    )

    # Current point in 2D
    fig.add_trace(
        go.Scatter(
            x=[x1_current], y=[x2_current],
            mode="markers",
            marker=dict(color='white',size=10),
            name="current (x1,x2)"
        ),
        row=1, col=2
    )

    # Boundary line in 2D
    if bx is not None:
        fig.add_trace(
            go.Scatter(
                x=bx, y=by,
                mode="lines",
                line=dict(width=4),
                name="boundary net=0"
            ),
            row=1, col=2
        )

    fig.update_scenes(
        xaxis_title="x1",
        yaxis_title="x2",
        zaxis_title="a",
        row=1, col=1
    )

    fig.update_xaxes(title_text="x1", range=[xlim[0], xlim[1]], row=1, col=2)
    fig.update_yaxes(title_text="x2", range=[ylim[0], ylim[1]], row=1, col=2)

    fig.update_layout(
        width=1100,
        height=520,
        margin=dict(l=10, r=10, t=50, b=10),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0.0)
    )

    display(fig)
    return fig

# =========================
# Main demo function (2D input only)
# - Prints net, a, loss, gradients
# - Shows activation curve + 3D surface+boundary+contour
# =========================
def neuron_demo_v3(
    x1=0.5, x2=0.5,
    w1=1.0, w2=1.0,
    b=0.0,
    activation="sigmoid",
    leaky_alpha=0.1,
    show_boundary=True,
    show_loss=True,
    t=0.0,
    grid_n=70
):
    # Forward pass (2D)
    x = np.array([x1, x2], dtype=float)
    w = np.array([w1, w2], dtype=float)
    net = float(np.dot(w, x) + b)
    a = float(apply_act(net, activation, leaky_alpha=leaky_alpha))

    # Loss and gradients (single sample): L = 1/2 (a - t)^2
    if show_loss:
        L = 0.5 * (a - t) ** 2
        dL_da = (a - t)
        da_dnet = float(apply_dact(net, activation, leaky_alpha=leaky_alpha))
        dL_dnet = dL_da * da_dnet

        dL_dw = dL_dnet * x
        dL_db = dL_dnet
    else:
        L = None
        dL_da = da_dnet = dL_dnet = None
        dL_dw = np.array([np.nan, np.nan])
        dL_db = np.nan

    # Print summary + gradients
    precision=2
    # print("-" * 72)
    # print(f"x = [{x1:.3f}, {x2:.3f}] ; w = [{w1:.3f}, {w2:.3f}],  b = {b:.3f}")
    # print(f"net = w·x + b = {net:.6f}  a = f(net) = {a:.6f}")
    # print(f"activation = {activation}" if activation != "leaky ReLU" else f"activation = leaky ReLU(leaky_alpha={leaky_alpha:.3f})")

    display_named_matrix({r"\mathbf{X}": {"value": x, "precision": precision,"one_d_as":"row"}})
    display_named_matrix({r"\mathbf{W}": {"value": w, "precision": precision, "one_d_as":"column"},
                          r"\mathbf{b}": {"value": b, "precision": precision}})
    display_named_matrix({
        r"\mathbf{net}": {"value": net, "precision": precision},
        r"\mathbf{actual}": {"value": a, "precision": precision},
        r"\mathbf{target}": {"value": t, "precision": precision},
        r"\mathbf{Loss}": {"value": L, "precision": precision},
        }),


    # display_named_matrix(w, name="W", precision=precision,one_d_as="column")
    # display_named_matrix([
    # (r"\mathbf{A}", x),
    # (r"\mathbf{B}", w, one_d_as="column"),
    # (r"\mathbf{C}", b),
    # ])
#     display_named_matrix([
#     (r"\mathbf{w}",w),
#     (r"\mathbf{b}", b, one_d_as="column"),
# ])
    # display_named_matrix([(w, name="W", precision=precision,one_d_as="column")])
    # display_named_matrix([
    #     (net, name="net", precision=precision),
    #     (a, name="a", precision=precision)
    # ])

    if show_loss:
      display_named_matrix({
        r"\mathbf{target}": {"value": t, "precision": precision},
        r"\mathbf{Loss}": {"value": L, "precision": precision},
        r"\frac{\partial \mathcal{L}}{\partial \mathbf{W}}_{1}" : {"value": dL_dw[0], "precision": precision},
        r"\frac{\partial \mathcal{L}}{\partial \mathbf{W}}_{2}" : {"value": dL_dw[1], "precision": precision},
        r"\frac{\partial \mathcal{L}}{\partial \mathbf{b}}" : {"value": dL_db, "precision": precision},
        })



        # print(f"target t = {t:.3f}  actual = {a:.6f}  Loss L = 1/2 (a - t)^2 = {L:.6f}")
        # print(f"dL/da = {dL_da:.6f},  da/dnet = {da_dnet:.6f},  dL/dnet = {dL_dnet:.6f}")
        # print(f"Gradients: dL/dw1 = {dL_dw[0]: .6f} , dL/dw2 = {dL_dw[1]: .6f} , dL/db  = {dL_db: .6f}")
    else:
        print("Loss/gradients are disabled (toggle 'show loss' to enable).")

    # Plot 2: 3D surface + boundary + 2D contour
    plot_output_surface_plotly(
        w1=w1, w2=w2, b=b,
        activation=activation, leaky_alpha=leaky_alpha,
        x1_current=x1, x2_current=x2, a_current=a,
        show_boundary=show_boundary,
        grid_n=int(grid_n),
    )

# =========================
# UI: x sliders in one row, w sliders (and b) in one row (2D only)
# =========================
slider_layout = Layout(width="260px")

# X sliders (2D)
x1_s = FloatSlider(value=0.5, min=-2, max=2, step=0.01, description="x1", layout=slider_layout)
x2_s = FloatSlider(value=0.5, min=-2, max=2, step=0.01, description="x2", layout=slider_layout)
x_row = HBox([x1_s, x2_s])

# W sliders (2D) + b on same row
w1_s = FloatSlider(value=1.0, min=-5, max=5, step=0.01, description="w1", layout=slider_layout)
w2_s = FloatSlider(value=1.0, min=-5, max=5, step=0.01, description="w2", layout=slider_layout)
b_s  = FloatSlider(value=0.0, min=-5, max=5, step=0.01, description="b",  layout=slider_layout)
w_row = HBox([w1_s, w2_s, b_s])

# Other controls
act_d = Dropdown(options=list(ACTS.keys()), value="sigmoid", description="act")
leaky_alpha_s = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description="leaky α", layout=slider_layout)
show_boundary_c = Checkbox(value=True, description="show boundary (net=0)")
show_loss_c = Checkbox(value=True, description="show loss")
t_s = FloatSlider(value=0.0, min=-2, max=2, step=0.01, description="target t", layout=slider_layout)
grid_n_s = FloatSlider(value=70, min=5, max=120, step=5, description="Plot Resolution", layout=slider_layout)

other_controls = HBox([act_d, leaky_alpha_s, show_boundary_c, show_loss_c, t_s, grid_n_s])

# Buttons + status
btn_reset_all = Button(description="Reset all")
btn_gd_step = Button(description="1-step GD", button_style="primary")
status = Label(value="")
btn_row = HBox([btn_reset_all, btn_gd_step, status])

eta_s = FloatSlider(value=0.10, min=0.0, max=2.0, step=0.01, description="η (lr)", layout=slider_layout)

# Wire output
out = interactive_output(
    neuron_demo_v3,
    {
        "x1": x1_s, "x2": x2_s,
        "w1": w1_s, "w2": w2_s,
        "b": b_s,
        "activation": act_d,
        "leaky_alpha": leaky_alpha_s,
        "show_boundary": show_boundary_c,
        "show_loss": show_loss_c,
        "t": t_s,
        "grid_n": grid_n_s,
    }
)

# Defaults
DEFAULT_X = (0.5, 0.5)
DEFAULT_WB = (1.0, 1.0, 0.0)

def reset_x(_):
    x1_s.value, x2_s.value = DEFAULT_X
    status.value = "x reset"

def reset_wb(_):
    w1_s.value, w2_s.value, b_s.value = DEFAULT_WB
    status.value = "w,b reset"

def reset_all(_):
    reset_x(None)
    reset_wb(None)
    act_d.value = "sigmoid"
    leaky_alpha_s.value = 0.1
    show_boundary_c.value = True
    show_loss_c.value = True
    t_s.value = 0.0
    eta_s.value = 0.10
    grid_n_s.value = 70
    status.value = "all reset"

btn_reset_all.on_click(reset_all)

# 1-step Gradient Descent update (single sample)
def gd_step(_):
    if not show_loss_c.value:
        status.value = "Enable 'show loss' for GD"
        return

    x = np.array([x1_s.value, x2_s.value], dtype=float)
    w = np.array([w1_s.value, w2_s.value], dtype=float)
    b = float(b_s.value)

    net = float(np.dot(w, x) + b)
    a = float(apply_act(net, act_d.value, leaky_alpha=leaky_alpha_s.value))

    t = float(t_s.value)
    dL_da = (a - t)
    da_dnet = float(apply_dact(net, act_d.value, leaky_alpha=leaky_alpha_s.value))
    dL_dnet = dL_da * da_dnet

    dL_dw = dL_dnet * x
    dL_db = dL_dnet

    eta = float(eta_s.value)

    w_new = w - eta * dL_dw
    b_new = b - eta * dL_db

    w1_s.value = float(np.clip(w_new[0], w1_s.min, w1_s.max))
    w2_s.value = float(np.clip(w_new[1], w2_s.min, w2_s.max))
    b_s.value  = float(np.clip(b_new, b_s.min, b_s.max))

    status.value = f"GD step: η={eta:.2f}"

btn_gd_step.on_click(gd_step)

controls = VBox([x_row, w_row, btn_row, other_controls, eta_s])
display(controls, out)



Output()